In [1]:
%config IPCompleter.greedy=True

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

In [3]:
xcelLocation = "../xcels/"
!export xcelLocation="../xcels/"

In [4]:
!ls $xcelLocation | grep ".xlsx" > xlFiles
tmp = !cat xlFiles
names = [name[:-5] for name in tmp]

In [5]:
def convert(xcelLocation, xlFileName, returnAllMode=False):
    xl = None
    try:
        if (not Path(xcelLocation + xlFileName + '.csv').is_file()) or returnAllMode:
            xl = pd.read_excel(xcelLocation + xlFileName + ".xlsx", index_col=0,
                               header=[0], skiprows=[0,1])
            xl.to_csv(xcelLocation + xlFileName + '.csv', encoding='utf-8', index=False)
    except:
        print(xlFileName)
    finally:
        return xl

In [6]:
allDFs = []
allDFNames = []
for name in names:
    tmp = convert(xcelLocation, name, returnAllMode=True)
    if tmp is not None:
        allDFs.append(tmp.copy())
        allDFNames.append(name)

1380-10-2
1380-1-11
1380-11-29
1380-1-14
1380-1-18
1380-1-19
1380-1-20
1380-12-14
1380-12-19
1380-1-21
1380-12-20
1380-1-22
1380-1-25
1380-12-5
1380-1-26
1380-1-27
1380-1-28
1380-1-29
1380-1-5
1380-1-6
1380-1-7
1380-1-8
1380-2-11
1380-2-12
1380-2-17
1380-2-18
1380-2-19
1380-2-22
1380-2-23
1380-2-25
1380-2-26
1380-2-30
1380-2-31
1380-2-3
1380-2-4
1380-2-5
1380-3-22
1380-3-23
1380-3-29
1380-3-5
1380-3-6
1380-3-9
1380-4-19
1380-4-2
1380-6-11
1380-8-15
1380-8-28
1380-9-10
1380-9-14
1380-9-18
1380-9-7
1381-10-4
1381-1-10
1381-11-19
1381-11-1
1381-11-28
1381-11-30
1381-11-5
1381-12-11
1381-12-19
1381-12-6
1381-2-14
1381-2-18
1381-3-21
1381-3-25
1381-3-5
1381-4-10
1381-4-23
1381-4-31
1381-5-22
1381-6-11
1381-6-17
1381-6-31
1381-6-9
1381-7-24
1381-8-11
1381-8-15
1381-8-22
1381-9-19
1381-9-24
1381-9-30
1382-10-24
1382-11-19
1382-12-17
1382-12-20
1382-12-26
1382-12-27
1382-12-2
1382-12-5
1382-2-20
1382-3-12
1382-3-17
1382-3-28
1382-3-5
1382-4-24
1382-4-25
1382-4-8
1382-5-4
1382-6-22
1382-6-24
13

In [15]:
print(len(allDFs))
def makeMasterTable():
    for index, df in enumerate(allDFs):
        year, month, day = allDFNames[index].split("-")
        yearlist = np.full(len(df), year).tolist()
        monthlist = np.full(len(df), month).tolist()
        daylist = np.full(len(df), day).tolist()
        df["year"] = yearlist
        df["month"] = monthlist
        df["day"] = daylist
    xl = pd.concat(allDFs, keys=allDFNames, ignore_index=True)
    xl.to_csv(xcelLocation + 'master.csv', encoding='utf-8', index=False)
    return xl
allDF = makeMasterTable()

6612


In [17]:
allDF.loc[allDF["نام"] == "شير و گوشت زاگرس شهركرد"]

,نام,تعداد,حجم,ارزش,دیروز,اولین,آخرین معامله - مقدار,آخرین معامله - تغییر,آخرین معامله - درصد,قیمت پایانی - مقدار,قیمت پایانی - تغییر,قیمت پایانی - درصد,کمترین,بیشترین,year,month,day
915830,شير و گوشت زاگرس شهركرد,112569,16000000,55846786595,1000.0,3400.0,3500.0,2500.0,250.00,3490.0,2490.0,249.00,3400.0,3500.0,1396,10,26
916361,شير و گوشت زاگرس شهركرد,1,1000000,3490000000,1000.0,3490.0,3490.0,2490.0,249.00,3490.0,2490.0,249.00,3490.0,3490.0,1396,10,26
916397,شير و گوشت زاگرس شهركرد,1317,458178,1678764192,3490.0,3664.0,3664.0,174.0,4.99,3664.0,174.0,4.99,3664.0,3664.0,1396,10,27
917599,شير و گوشت زاگرس شهركرد,774,389357,1497856379,3664.0,3847.0,3847.0,183.0,4.99,3847.0,183.0,4.99,3847.0,3847.0,1396,10,30
921116,شير و گوشت زاگرس شهركرد,1632,1521721,8423550687,5291.0,5500.0,5555.0,264.0,4.99,5536.0,245.0,4.63,5400.0,5555.0,1396,11,10
921757,شير و گوشت زاگرس شهركرد,321,732319,4255385145,5536.0,5812.0,5809.0,273.0,4.93,5811.0,275.0,4.97,5705.0,5812.0,1396,11,11
922286,شير و گوشت زاگرس شهركرد,6403,4949729,29629127883,5811.0,6101.0,5850.0,39.0,0.67,5986.0,175.0,3.01,5796.0,6406.0,1396,11,14
922890,شير و گوشت زاگرس شهركرد,4910,2830354,16195399339,5986.0,5751.0,5687.0,-299.0,-4.99,5722.0,-264.0,-4.41,5687.0,5840.0,1396,11,15
923458,شير و گوشت زاگرس شهركرد,5862,3138878,17314643296,5722.0,5510.0,5603.0,-119.0,-2.08,5516.0,-206.0,-3.60,5436.0,5636.0,1396,11,16
924055,شير و گوشت زاگرس شهركرد,3008,2367520,13692615317,5516.0,5600.0,5791.0,275.0,4.99,5784.0,268.0,4.86,5555.0,5791.0,1396,11,17
